In [2]:
import torch
import torch.nn.functional as F
# from confusion import plot_confusion_matrix
import plotly.express as px

In [3]:
def plot_confusion_matrix(idx2category, labels_all_finetune, logits_all_finetune):
    # modelnet_dict = torch.load("src/eval_data/modelnet_dict.pt")

    cat_list = list(idx2category.values())
    
    print("labels_all: ", labels_all_finetune.shape)
    
    _, pred = logits_all_finetune.topk(1, 1, True, True)
    pred = pred.reshape(-1)
    
    true_pred = torch.argwhere(pred == labels_all_finetune).reshape(-1)
    false_pred = torch.argwhere(pred != labels_all_finetune).reshape(-1)
    
    res = torch.zeros(len(cat_list), len(cat_list))
    for val in true_pred:
        idx = val.item()
        res[labels_all_finetune[idx], pred[idx]] += 1
    
    for val in false_pred:
        idx = val.item()
        res[labels_all_finetune[idx], pred[idx]] += 1
        # print(labels_all[idx], pred[idx])

    
    res_norm = F.softmax(res, dim=1)

    return res, res_norm

In [4]:
def get_idx2category():
    objaverse_dict = torch.load("src/eval_data/objaverse_dict.pt")
    return objaverse_dict["idx2category"]

In [5]:
# confusion_pairs = [(464, 409), (464, 861), (372, 510), (106, 548), (1019, 1020), (1019, 562)]
confusion_pairs = [(464, 464), (464, 409), (464, 861), (372, 372), (372, 510), (106, 106), (106, 548), (1019, 1019), (1019, 1020), (1019, 562)]
all_finetune_cats = [464, 409, 861, 372, 510, 106, 548, 1019, 1020, 562]

In [18]:
idx2category = get_idx2category()
cat_names = []
for cat in all_finetune_cats:
    cat_names.append(idx2category[cat])
cat_names

['goldfish',
 'fish',
 'salmon_(fish)',
 'earphone',
 'headset',
 'blazer',
 'jacket',
 'teakettle',
 'teapot',
 'kettle']

In [46]:
ov_labels_all_only_test = torch.load("exp/finetune_1layer@20240129-151703/results/ov_labels_all_finetune_test.pt")
ov_logits_all_only_test = torch.load("exp/finetune_1layer@20240129-151703/results/ov_logits_all_finetune_test.pt")

In [53]:
epoch = 60
ov_labels_all_finetune = torch.load(f"exp/finetune_1layer@20240129-151703/results/ov_labels_all_finetune_ov_test_all_epoch_{epoch}.pt")
ov_logits_all_finetune = torch.load(f"exp/finetune_1layer@20240129-151703/results/ov_logits_all_finetune_ov_test_all_epoch_{epoch}.pt")

idx2category = get_idx2category()
ov_res_finetune, ov_res_norm_finetune = plot_confusion_matrix(idx2category, ov_labels_all_finetune, ov_logits_all_finetune)
ov_res_baseline, ov_res_norm_baseline = plot_confusion_matrix(idx2category, ov_labels_all_only_test, ov_logits_all_only_test)

labels_all:  torch.Size([46205])
labels_all:  torch.Size([46205])


In [54]:
ov_res_baseline[all_finetune_cats].shape

torch.Size([10, 1156])

In [55]:
# ov_res_norm_finetune[all_finetune_cats][:, all_finetune_cats]

In [57]:
pred_diff = ov_res_finetune[all_finetune_cats][:, all_finetune_cats] - ov_res_baseline[all_finetune_cats][:, all_finetune_cats]
pred_diff

tensor([[  1.,  -5.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
        [  0.,  -4.,   1.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
        [  0.,  -7.,   4.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,  -6.,  -1.,   0.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,  -3.,  -2.,   0.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.,   0.,  -1., -19.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.,   0.,   1., -10.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.,   0.,   0.,   0.,  -1.,  -4.,   0.],
        [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   4.,  -8.,   0.],
        [  0.,   0.,   0.,   0.,   0.,   0.,   0.,  -5.,  -2.,   2.]])

In [25]:
(ov_res_finetune[all_finetune_cats] - ov_res_baseline[all_finetune_cats]).shape

torch.Size([10, 1156])

In [33]:
pred_diff_diag = torch.diag(torch.diag(pred_diff))
pred_diff_diag
# Positive is good, negative is bad

tensor([[  1.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
        [  0.,  -4.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   4.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,  -6.,   0.,   0.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.,  -2.,   0.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.,   0.,  -1.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.,   0.,   0., -10.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.,   0.,   0.,   0.,  -1.,   0.,   0.],
        [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,  -8.,   0.],
        [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   2.]])

In [44]:
fig = px.imshow(pred_diff_diag.numpy(), x=cat_names, y=cat_names, color_continuous_scale='blues_r')
fig.show()

In [38]:
pred_diff_no_diag = pred_diff - pred_diff_diag
pred_diff_no_diag
# Negative is good, positive is bad

tensor([[  0.,  -5.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   1.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
        [  0.,  -7.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.,  -1.,   0.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,  -3.,   0.,   0.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.,   0.,   0., -19.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.,   0.,   1.,   0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,  -4.,   0.],
        [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   4.,   0.,   0.],
        [  0.,   0.,   0.,   0.,   0.,   0.,   0.,  -5.,  -2.,   0.]])

In [45]:
fig = px.imshow(pred_diff_no_diag.numpy(), x=cat_names, y=cat_names, color_continuous_scale='blues_r')
fig.show()
# pred_diff.numpy()

In [30]:
ov_res_finetune[all_finetune_cats][:, all_finetune_cats]

tensor([[18., 32.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 3., 64.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  8., 10.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0., 22., 50.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  7., 54.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0., 13.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  1., 26.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  6., 55.,  3.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  5., 66.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  4., 46., 20.]])

In [24]:
pred_diff = ov_res_finetune[all_finetune_cats] - ov_res_baseline[all_finetune_cats]
fig = px.imshow(pred_diff.numpy(), color_continuous_scale='gray')
fig.show()